In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import *
from sklearn.metrics import *
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data = pd.read_csv('./datasets/mushroom.csv')

In [ ]:
data.info()

In [ ]:
data.drop(columns = ['veil-type'], inplace = True)

In [ ]:
from tqdm.contrib.concurrent import process_map
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
mask = data.isnull()
# class 使用 OneHotEncoder
data['class'] = OneHotEncoder().fit_transform(
    data['class'].values.reshape(-1, 1)).toarray()
# 其余使用 LabelEncoder 编码
for col in data.columns:
    if col != 'class':
        data[col] = LabelEncoder().fit_transform(data[col])

data.where(~mask, np.nan, inplace=True)


def test(n):
    global data
    df = pd.DataFrame(KNNImputer(
        n_neighbors=5).fit_transform(data), columns=df.columns)

    x = df.drop('class', axis=1)
    y = df['class']

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42)
    ada_model = AdaBoostClassifier()

    parameters = {
        'n_estimators': np.arange(150, 170, 10),
        'learning_rate': np.logspace(-1, 1, 10),
        'algorithm': ['SAMME']
    }

    grid_search = GridSearchCV(
        ada_model, parameters, n_jobs=-1, cv=5, verbose=1)
    grid_search.fit(x_train, y_train)

    ada_model = AdaBoostClassifier(n_estimators=grid_search.best_params_[
        'n_estimators'], learning_rate=grid_search.best_params_['learning_rate'])

    ada_model.fit(x_train, y_train)

    ada_pred = ada_model.predict(x_test)

    return f1_score(y_test, ada_pred)


scores = process_map(test, range(1, 95*2, 2))

# 画图
sns.lineplot(x=range(1, 95*2, 2), y=scores)
